**Praktis Model Selection**

1. Bagi kedua data ini menjadi data training dan data test dengan test_size=0.25.
2. Gunakan algoritma KNN sebagai model classifier.
3. Gunakan fungsi GridSearchCV untuk hyperparameter tuning dan model selection.
4. jumlah fold bebas!, gunakan scoring 'roc_auc'
5. Definisikan kombinasi hyperparameter untuk model selection dengan GridSearchCV. kombinasi Hyperparameter bebas, baca lagi dokumentasi KNN di link berikut https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html untuk memahami lagi jenis2 hyperparameter di algorithma KNN.
6. Latih model terhadap data training.
7. Apa hyperparameter terbaik untuk kombinasi hyperparameter kalian?
8. Berapa score validasi terbaik dari model tersebut?
9. Prediksi probabilitasi output dari model yang telah di buat terhadap data test. note : gunakan method .predict_proba() untuk menghasilkan output probabilitas
10. Perhatikan bahwa hasil prediksi ada 2, dimana masing2 adalah nilai probabilitas untuk setiap class label. Ambil nilai probabilitas pasien phositive meninggal dalam waktu kurang dari 5 tahun. note : gunakan bantuan attirubte .classes_ untuk mengetahui urutan label dari hasil prediksi probabilitas.
11. Berapa nilai score roc_auc untuk data test?
12. Apakah model anda termasuk baik atau overtting atau underfitting?
13. Ulangi tahap di atas namun kali ini menggunakan algoritma DecisionTreeClassifier dan kalian bisa menggunakan RandomizedSearchCV apabila process training lama. pelajari algoritma DecisionTreeClassifier di linkberikut : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=decisiontreeclassifier#sklearn.tree.DecisionTreeClassifier
14. Bandingkan scorenya dengan Algoritma KNN, mana yang lebih baik?

Import dataset

In [ ]:
import pandas as pd
import numpy as np

url = '../input/habermans-survival-data-set/haberman.csv'
list_cols = ['Age', "Patient's Years", "N_positive_ax", "survival_status"]
df = pd.read_csv(url, names=list_cols)

In [ ]:
df.head()

Menghitung value dari kolom 'survival_status'

In [ ]:
df['survival_status'].value_counts()

Menentukan label dan fitur serta membagi data menjadi data train dan data test

In [ ]:
X = df.drop(['survival_status'], axis=1)
y = df['survival_status']

# spliting daa
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21, stratify=y)

Menggunakan GridSearchCV untuk hyperparameter tuning dan model selection dan model algoritma KNN sebagai model classifier.

Menggunakan gscv untuk tuning hyperparameter model algoritma KNN dengan n array 5-50 dan weights 'distance' dan 'uniform'.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import confusion_matrix, classification_report
model_knn =  KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(5,50), 'weights':['distance','uniform']}
gscv= GridSearchCV(model_knn,param_grid=param_grid, scoring='roc_auc', cv=5)
#latih model
gscv.fit(X,y)

Mendapatkan hyperparameter terbaik untuk kombinasi hyperparameterMendapatkan

In [ ]:
gscv.best_params_

Score validasi terbaik

In [ ]:
gscv.best_score_

Prediksi probabilitasi output

In [ ]:
gscv.predict_proba(X_test)

Menampilkan hasil prediksi dimana masing2 adalah nilai probabilitas untuk setiap class label dan urutan label dari hasil prediksi probabilitas.

In [ ]:
gscv.classes_
gscv.predict_proba(X_test)[:,1]

In [ ]:
cv_score_1 = cross_validate(model_knn, X,y, scoring='roc_auc', cv=5, return_train_score=True)
cv_score_2 = cross_val_score(model_knn, X,y, scoring='roc_auc')
cv_score_1

In [ ]:
def knn_pred(k):
  model = KNeighborsClassifier(n_neighbors=k)
  score = cross_validate(model, X,y, cv=10, return_train_score=True)
  train_score = score['train_score'].mean()
  test_score = score['test_score'].mean()
  return train_score, test_score

In [ ]:
train_scores = []
test_scores = []

for k in range(2,100):
  train_score, test_score = knn_pred(k)
  train_scores.append(train_score)
  test_scores.append(test_score)

**Visualisasi score dari K data train dan data test**

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,8))

ax.plot(range(2,100), train_scores, marker='x', label='Train Score', color='g')
ax.plot(range(2,100), test_scores, marker='o', label='Test Score', c='b')

ax.set_xlabel('K')
ax.set_ylabel('score')

plt.legend()
plt.show()

dari visualisasi diatas didapatkan model ini memberikan performa/skor baik dengan k = 40

**Model Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
model_dt = DecisionTreeClassifier()
param_grid = {'splitter':['best','random'], 'criterion':['gini','entropy']}
rscv= RandomizedSearchCV(model_dt,param_distributions=param_grid,n_iter=100,scoring='roc_auc')
#latih model
rscv.fit(X,y)

Hyperparameter terbaik untuk kombinasi hyperparameter

In [ ]:
rscv.best_params_

Prediksi probabilitasi output

In [ ]:
rscv.predict_proba(X_test)

Menampilkan hasil prediksi dimana masing2 adalah nilai probabilitas untuk setiap class label dan urutan label dari hasil prediksi probabilitas.

In [ ]:
rscv.classes_
rscv.predict_proba(X_test)[:,1]

In [ ]:
cv_score_1_ = cross_validate(model_dt, X,y, scoring='roc_auc', cv=5, return_train_score=True)
cv_score_2_ = cross_val_score(model_dt, X,y, scoring='roc_auc')
cv_score_1_